# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the Yelp/yelp_review_full dataset | sentiment classification (1-5)
# https://huggingface.co/datasets/Yelp/yelp_review_full

splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("Yelp/yelp_review_full", split=splits))}

# Thin out the dataset to make it run faster for this project
for split in splits:
  ds[split] = ds[split].shuffle(42).select(range(500))

# Show the dataset
ds

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['label', 'text'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['label', 'text'],
     num_rows: 500
 })}

In [3]:
# Inspect the first example
ds["train"][0]

{'label': 4,
 'text': "I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!"}

## Pre-process datasets

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_dataset = {}
for split in splits:
  tokenized_dataset[split] = ds[split].map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True),
                                           batched=True)

# Show the tokenized_dataset["train"]
print(tokenized_dataset["train"])

# Show the first example of the tokenized training set
print(tokenized_dataset["train"][0]["input_ids"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 500
})
[101, 1045, 23899, 2023, 4744, 1012, 1045, 1005, 2310, 2042, 2000, 3919, 6328, 2073, 1045, 9811, 2000, 2022, 1037, 6627, 7309, 3061, 1999, 2240, 1010, 6167, 6670, 5581, 7167, 1010, 1998, 1997, 2607, 1996, 7500, 1005, 1055, 3006, 1012, 1996, 15220, 2024, 2061, 2061, 7078, 7746, 1012, 1996, 3954, 2003, 3565, 5379, 1998, 2002, 3084, 2169, 4605, 2011, 2192, 2007, 2019, 9788, 3815, 1997, 6620, 1012, 2017, 10657, 4521, 2182, 4364, 999, 999, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Load HF Foundation model and freeze base parameters

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 5,
    id2label = {1 : "worst", 2 : "bad", 3 : "average", 4 : "good", 5 : "great"},
    label2id = {"worst" : 1, "bad" : 2, "average" : 3, "good" : 4, "great" : 5}
  )

# Freeze all the parameters of the base model
for param in model.base_model.parameters():
  param.requires_grad = False

print(model)
print(model.classifier)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [6]:
# classification metrics
import numpy as np

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return {"accuracy": (predictions == labels).mean()}

In [7]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir = "./results",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=1,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.603649,0.190000


{'eval_loss': 1.6036489009857178,
 'eval_accuracy': 0.19,
 'eval_runtime': 7.8751,
 'eval_samples_per_second': 63.491,
 'eval_steps_per_second': 31.746,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
from peft import LoraConfig, get_peft_model, TaskType

In [9]:
# https://huggingface.co/docs/peft/main/en/conceptual_guides/lora

config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["classifier"], # The modules (eg. attention blocks) to apply the LoRA update matrices
    bias="none"
)

# create a PEFT model
lora_peft_model = get_peft_model(model, config) # model is defined above
lora_peft_model.print_trainable_parameters()

trainable params: 1,195,058 || all params: 67,557,938 || trainable%: 1.7689379447904405


In [15]:
# HuggingFace Trainer
ft_trainer = Trainer(
    model=lora_peft_model,
    
    args=TrainingArguments(
        output_dir = "./ft_results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_safetensors=False,
    ),

    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# train model
ft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.555547,0.386000
2,No log,1.546465,0.382000
3,No log,1.541146,0.382000
4,No log,1.539061,0.378000


Checkpoint destination directory ./ft_results/checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=252, training_loss=1.5374070424882194, metrics={'train_runtime': 67.396, 'train_samples_per_second': 29.675, 'train_steps_per_second': 3.739, 'total_flos': 268639186944000.0, 'train_loss': 1.5374070424882194, 'epoch': 4.0})

In [16]:
ft_trainer.evaluate()

{'eval_loss': 1.5390610694885254,
 'eval_accuracy': 0.378,
 'eval_runtime': 8.0893,
 'eval_samples_per_second': 61.81,
 'eval_steps_per_second': 7.788,
 'epoch': 4.0}

## Save a trained PEFT Model

In [40]:
# Save the PEFT model
# https://huggingface.co/docs/peft/main/en/package_reference/lora#peft.LoraModel.merge_and_unload

lora_peft_model.save_pretrained("lora_peft_model", save_config=True, save_adapter=True)
lora_peft_model.merge_and_unload()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [41]:
from peft import PeftModel, PeftConfig, AutoPeftModelForSequenceClassification

In [47]:
config2 = PeftConfig.from_pretrained("lora_peft_model", num_labels=5)
model2 = AutoPeftModelForSequenceClassification.from_pretrained("lora_peft_model", num_labels=5)
model2

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
             

In [48]:
tokenizer2 = AutoTokenizer.from_pretrained(config2.base_model_name_or_path)

In [49]:
# Generate
inputs = tokenizer2("Best methods to regain momentum is ", truncation=True, return_tensors="pt")
outputs = model2.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer2.batch_decode(outputs))

TypeError: The current model class (DistilBertForSequenceClassification) is not compatible with `.generate()`, as it doesn't have a language model head.

## Evaluate the fine-tuned model

In [ ]:
# HuggingFace Trainer
ft_trainer2 = Trainer(
    model=model2,
    args=TrainingArguments(
        output_dir = "./results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_safetensors=False,
    ),

    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# train model
ft_trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.578098,0.356000
2,No log,1.570668,0.350000
3,No log,1.566048,0.352000


In [ ]:
ft_trainer2.evaluate()